In [ ]:
# Imports
from sklearn import datasets
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import precision_score, accuracy_score, confusion_matrix, classification_report
from sklearn import tree
import numpy as np
import sys
from sklearn.model_selection import KFold
from tqdm import tqdm
from scipy.stats import sem
import joblib
import hashlib
import sqlite3
from math import sin, cos, sqrt, atan2, radians, floor
from matplotlib.pyplot import figure
import os

# Methods
def prepare_data(dataset, sample_frac=1):
    df = dataset
    temp = pd.DataFrame()
#     temp["dest"] = df['arrival_point']
#     temp["dest_hash"] = df['arrival_point'].apply(lambda x: my_hash(str(x)) if not None else None)
    temp["dest_latitude"] = df['arrival_point'].apply(lambda x: df_airport_data["latitude"][x] if (x in df_airport_data.index) else None)
    temp["dest_longitude"] = df['arrival_point'].apply(lambda x: df_airport_data["longitude"][x] if (x in df_airport_data.index) else None)
#     temp["dest_square"] = temp.apply(lambda x: get_grid_square(x["dest_latitude"], x["dest_longitude"]) if not None and not np.isnan(x["dest_latitude"]) else None, axis=1)
#     temp["dest_has_ils"] = df['arrival_point'].apply(lambda x: "ILS" in df_airport_data["approaches"][x] if (x in df_airport_data.index and isinstance(df_airport_data["approaches"][x], str)) else False).astype(int)
#     temp["dest_longest_runway"] = df["arrival_point"].apply(lambda x: df_airport_data["longest_runway"][x] if (x in df_airport_data.index) else None)
#     temp["dest_weather"] = df["destination_airport_weather"].map({'Flight category unknown': None, 'IFR': 0, 'low IFR': 1, 'Marginal VFR': 2, 'VFR': 3})
#     temp["aircraft_type_icao"] = df['aircraft_type_icao']
#     temp["aircraft_type_icao_hash"] = df['aircraft_type_icao'].apply(lambda x: my_hash(str(x)) if not None else None)
#     temp["aircraft_max_landing_weight"] = df['aircraft_type_icao'].apply(lambda x: df_aircraft_data["aircraft_max_landing_weight"][x] if (x in df_aircraft_data.index) else None)
#     temp["aircraft_weight_class"] = temp['aircraft_max_landing_weight'].apply(lambda x: get_aircraft_weight_class(x))
    temp["wake_turbulence"] = df['wake_turbulence_category'].map({"L": 0, "LIGHT": 0, "Light": 0, "M": 1, "MEDIUM": 1, "Medium": 1, "L/M": 1, "H": 2, "Heavy": 2, "J": 3})
#     temp["uses_ifr"] = df["flight_rules"].map({'IFR': 1, 'VFR': 0})
#     temp["received_at_mm"] = df["received_at"].apply(lambda x: x[14:16]).astype("int16")
#     temp["received_at_HH"] = df["received_at"].apply(lambda x: x[11:13]).astype("int16")
#     temp["received_at_DD"] = df["received_at"].apply(lambda x: x[8:10]).astype("int16")
#     temp["received_at_MM"] = df["received_at"].apply(lambda x: x[5:7]).astype("int16")
#     temp["received_at_YY"] = df["received_at"].apply(lambda x: x[0:4]).astype("int16")
    temp["alternate_aerodrome"] = df["alternate_aerodrome"]
#     temp["alternate_square"] = temp.apply(lambda x: get_grid_square(df_airport_data["latitude"][x["alternate_aerodrome"]], df_airport_data["longitude"][x["alternate_aerodrome"]]) if (x["alternate_aerodrome"] in df_airport_data.index) and not np.isnan(df_airport_data["latitude"][x["alternate_aerodrome"]]) else None, axis=1)
    
    temp = temp.dropna()
    temp = temp.sample(frac=sample_frac, random_state=0)
    temp = temp.reset_index(drop=True)
    df = temp
    
    X = df[[
#              "dest",
#              "dest_hash",
             "dest_latitude",
             "dest_longitude",
#              "dest_square",
#              "dest_has_ils",
#              "dest_longest_runway",
#              "dest_weather",
#              "aircraft_type_icao",
#              "aircraft_type_icao_hash",
#              "aircraft_max_landing_weight",
             "wake_turbulence",
#              "aircraft_weight_class",
#              "uses_ifr",
#              "received_at_mm",
#              "received_at_HH",
#              "received_at_DD",
#              "received_at_MM",
#              "received_at_YY",
           ]]
    y = df["alternate_aerodrome"]
    
    return df, X, y
    

def myround(x, base):
    # Rounding to nearest integer n
    return base * (x // base)

def cyclic_encode(data, col, max_val):
    # Encoding cyclic data (time and date) to make the model consider their cyclic nature.
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    del data[col]
    return data

def my_hash(string):
    # Consistent hash method
    return int(str(int(hashlib.sha256(string.encode('utf-8')).hexdigest(), base=16))[:10])

def get_grid_square(lat, lon):
    lat += 90
    lon += 180
    
    lat_rounded = floor(lat)
    lon_rounded = floor(lon)
    
#     square_id = 360 * lat_rounded + lon_rounded
    square_id = 180 * lon_rounded + lat_rounded
    return square_id

def get_coords_from_square(square_id):
#     lat_rounded = square_id // 360
#     lon_rounded = square_id - 360 * lat_rounded
    
    lon_rounded = square_id // 180
    lat_rounded = square_id - 180 * lon_rounded
    
    return [lat_rounded - 90, lon_rounded - 180]
    

def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

def get_flight_plan_data(conn):
    query = "SELECT * FROM '1_mio_alternate_data' WHERE arrival_point LIKE 'K%' OR arrival_point LIKE 'E%' OR arrival_point LIKE 'L%' OR arrival_point LIKE 'C%'"

    cur = conn.cursor()
    cur.execute(query)
    
    columns_names = list(map(lambda x: x[0], cur.description))
    rows = cur.fetchall()
    
    return rows, columns_names

def get_airport_coords(conn):
    query = "SELECT ICAO, Latitude as latitude, Longitude AS longitude, LongestRunway AS longest_runway, HasILSApproach AS has_ils, HasRNAVApproach AS has_rnav, HasLocalizerApproach AS has_loc FROM (SELECT * FROM Airport INNER JOIN Point P on Airport.Point = P.Id) WHERE ICAO != ''"

    cur = conn.cursor()
    cur.execute(query)
    
    columns_names = list(map(lambda x: x[0], cur.description))
    rows = cur.fetchall()
    
    return rows, columns_names

def get_more_airport_data(conn):
    query = "SELECT alternates_airports_displayed_to_user as alternates, alternates_airport_is_towered_displayed_to_user AS has_tower, alternates_airport_approaches_displayed_to_user AS approaches FROM '1_mio_alternate_data'"

    cur = conn.cursor()
    cur.execute(query)
    
    columns_names = list(map(lambda x: x[0], cur.description))
    rows = cur.fetchall()
    
    return rows, columns_names

def get_aircraft_data(conn):
    query = "SELECT aircraft_type_icao, MAX(aircraft_max_landing_weight) AS aircraft_max_landing_weight, MIN(aircraft_min_runway_length) AS aircraft_min_runway_length FROM 'airplane_data (corrected with missing aircrafts) (from CSV)' GROUP BY (aircraft_type_icao)"

    cur = conn.cursor()
    cur.execute(query)
    
    columns_names = list(map(lambda x: x[0], cur.description))
    rows = cur.fetchall()
    
    return rows, columns_names

def get_aircraft_weight_class(weight):
    if weight < 15500:
        return 0
    elif weight < 300000:
        return 1
    elif weight < 1234588:
        return 2
    else:
        return 3

def calc_coord_distance(lat1, lon1, lat2, lon2):
    # Calculates distance between two points in km
    
    R = 6373.0    # Approximate radius of earth in km

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance

def airport_attractiveness_sort(dest, alternates):
    sorted_alternates = []
    for alternate_icao in alternates:
        if alternate_icao == dest.icao:
            continue
        sorted_alternates.append(Alternate(alternate_icao, dest))
    
    sorted_alternates = sorted(sorted_alternates, key=lambda x: x.distance)
    sorted_alternates = sorted(sorted_alternates, key=lambda x: x.has_tower, reverse=True)
    
    return sorted_alternates

class Airport:
    def __init__(self, icao):
        self.icao = icao
        self.lat = df_airport_data.loc[icao]["latitude"]
        self.lon = df_airport_data.loc[icao]["longitude"]
        self.has_tower = df_airport_data.loc[icao]["has_tower"]
        self.approaches = df_airport_data.loc[icao]["approaches"]
        self.has_ils = isinstance(self.approaches, str) and "ILS" in self.approaches
    
    def __str__(self):
        return "  - " + str(self.icao) + "{:10.3f}".format(self.lat) + "{:10.3f}".format(self.lon) + "\tTower:" + str(self.has_tower) + "   Approaches:" + str(self.approaches)

class Alternate(Airport):
    def __init__(self, icao, dest_airport):
        super().__init__(icao)
        self.distance = round(calc_coord_distance(dest_airport.lat, dest_airport.lon, self.lat, self.lon), 3)
    
    def __str__(self):
        return "  - " + str(self.icao) + "{:10.3f}".format(self.distance) + "km" + "\tTower:" + str(self.has_tower) + "   Approaches:" + str(self.approaches)

### Test methods

In [ ]:
def accuracy_score_in_top_ten(X_test, y_test, model, test_size):
    # X_test and y_test need to be respectively a Pandas.DataFrame and a Pandas.Series
    X_test_list = X_test.values.tolist()
    y_test_list = y_test.tolist()
    
    n_correct = 0
    for i in range(test_size):
        if i % 200 == 0 and i != 0:
            progress = round(i / test_size * 100, 2)
            print("                                                                                    ", end="\r")
            print(str(progress) + "%    ", n_correct / i, end="\r")
            
        attributes = X_test_list[i]
        predicted_alternates = model.classes_[model.predict_proba([attributes])[0].argsort()[-10:][::-1]]
    
        if (y_test_list[i] in predicted_alternates):
            n_correct += 1
    print("                                                                                    ", end="\r")
    final_acc_top10 = n_correct / test_size
    print(final_acc_top10)
    return final_acc_top10

def accuracy_score_own(X_test, y_test, model, test_size):
    # X_test and y_test need to be respectively a Pandas.DataFrame and a Pandas.Series
    X_test_list = X_test.values.tolist()
    y_test_list = y_test.tolist()
    
    n_correct = 0
    for i in range(test_size):
        if i % 200 == 0 and i != 0:
            progress = round(i / test_size * 100, 2)
            print("                                                                                    ", end="\r")
            print(str(progress) + "%    ", n_correct / i, end="\r")
            
            
        attributes = X_test_list[i]
        predicted_alternates = model.classes_[model.predict_proba([attributes])[0].argsort()[-1:][::-1]]
    
        if (y_test_list[i] in predicted_alternates):
            n_correct += 1
    final_acc_own = n_correct / test_size
    print("                                                                                    ", end="\r")
    print(final_acc_own)
    return final_acc_own

### Initialize airport data

In [ ]:
# Fetching airport coordinates
conn = create_connection("data\PE2014A\PE2014A.sqlite")
rows, column_names = get_airport_coords(conn)
df_airport_coords = pd.DataFrame(rows, columns=column_names)
df_airport_coords = df_airport_coords.set_index("ICAO")

# Fethcing more airport features (tower, approaches) 
rows, column_names = get_more_airport_data(conn)
df_airport_data = pd.DataFrame(rows, columns=column_names)
for column in df_airport_data.columns:
    df_airport_data[column] = df_airport_data[column].astype(str)

airport_dict = {}

for i, row in df_airport_data.iterrows():
    if i % 10000 == 0:
        progress = round(i / len(df_airport_data.index) * 100, 2)
        print(str(progress) + "%  ", end="\r")

    alternates = row["alternates"].split(",")
    towers = row["has_tower"].split(",")
    approaches = row["approaches"].replace(", ", ";").split(",")

    for i in range(len(alternates)):
        if alternates[i] not in airport_dict.keys():
            value = []
            value.append(1 if towers[i] == "Towered" else 0)
            value.append(approaches[i])

            airport_dict[alternates[i]] = value

print("100%  ", end="\r")
df_airport_data = pd.DataFrame.from_dict(airport_dict, orient='index', columns=["has_tower", "approaches"])

# Combining dataframes to make the one and only airport_data
df_airport_data = pd.concat([df_airport_coords, df_airport_data], axis=1).sort_index()

### Initialize Aircraft data

In [ ]:
conn = create_connection("data\PE2014A\PE2014A.sqlite")

rows, column_names = get_aircraft_data(conn)
df_aircraft_data = pd.DataFrame(rows, columns=column_names)
df_aircraft_data = df_aircraft_data.set_index("aircraft_type_icao")

### Load dataset

In [ ]:
conn = create_connection("data\PE2014A\PE2014A.sqlite")
rows, column_names = get_flight_plan_data(conn)
dataset = pd.DataFrame(rows, columns=column_names)

dataframe, X, y = prepare_data(dataset, 1)

### Further preprocessing

In [ ]:
# minMax = MinMaxScaler()
# std_scaler = StandardScaler()

# LabelEncoder
if "dest" in X.columns:
   X["dest"] = LabelEncoder().fit_transform(X["dest"]).tolist()
if "aircraft_type_icao" in X.columns:
   X["aircraft_type_icao"] = LabelEncoder().fit_transform(X["aircraft_type_icao"]).tolist()
# if "wake_turbulence" in X.columns:
#     X_waket = pd.DataFrame(X["wake_turbulence"])
# #     X_runw = pd.DataFrame(X["dest_longest_runway"])
#     X_wake = minMax.fit_transform(X_waket)
# #     X_rnw = minMax.fit_transform(X_runw)
#     del X["wake_turbulence"]
# #     del X["dest_longest_runway"]

#Normalize
# X = pd.DataFrame(std_scaler.fit_transform(X), columns=X.columns)
# joblib.dump(std_scaler, 'std_scaler.bin', compress=True)

# X = pd.DataFrame(minMax.fit_transform(X), columns=X.columns)
# joblib.dump(minMax, 'minMax.bin', compress=True)


# X = pd.DataFrame(std_scaler.fit_transform(X), columns=X.columns)
# # X["dest_longest_runway"] = X_rnw
# X["wake_turbulence"] = X_wake
# joblib.dump(minMax, 'minMax_lat_long_rnw_wake.bin', compress=True)
# joblib.dump(std_scaler, 'stdScaler_lat_long_rnw_wake.bin', compress=True)

In [ ]:
# split into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

### Training classifier

In [ ]:
k_data = []
test_acc_std = None
train_acc_std = None
test_acc_top10 = None
train_acc_top10 = None
size = None

testArray = [1]
# testArray = [1]
for k in testArray:
    print("\nStarting Random Forest with K value = : ", k)
    model = RandomForestClassifier(criterion="entropy", max_depth=13, n_estimators=40, max_features=2)
    model.fit(X_train, y_train)
    
    print("Starting test_acc_std")
    test_acc_std = accuracy_score_own(X_test, y_test, model, 30000)
#     print(test_acc_std)
    
    print("Starting train_acc_std")
    train_acc_std = accuracy_score_own(X_train, y_train, model, 30000)
#     print(train_acc_std)
    
    print("Starting test_acc_top10")
    test_acc_top10 = accuracy_score_in_top_ten(X_test, y_test, model, 30000)
#     print(test_acc_top10)
    
    print("Starting train_acc_top10")
    train_acc_top10 = accuracy_score_in_top_ten(X_train, y_train, model, 30000)
#     print(train_acc_top10)

    
#     # Calculating size
#     file_name =  "./random_forest_nestimators_" + str(k) + ".joblib"
#     joblib.dump(model, file_name)
#     size = os.path.getsize(file_name) / (1024 ** 2)  # Converting to MB
#     os.remove(file_name)
#     print(size)
    
    k_data.append({"k": k,
                  "test_acc_std": test_acc_std,
                  "train_acc_std": train_acc_std,
                  "test_acc_top10": test_acc_top10,
                  "train_acc_top10": train_acc_top10,
                  "size": size})

print(k_data)